# csv des labels numériques des lycee

## Import data  from api

In [4]:
#importing necessary packages
import pandas as pd
import requests
import json

#creating an empty list
data_list = []

#creating a loop to get the data from all the available pages
for i in range(0, 10000, 1000):
    #creating the url
    url = 'https://data.occitanie.education.gouv.fr/api/records/1.0/search/?dataset=fr-en-occitanie-label-numerique-lycee&q=&rows=1000&start='+str(i)+'&facet=annee&facet=rne&facet=departement&facet=label'
    #making the request
    response = requests.get(url)
    #converting the response to json
    data = response.json()
    #appending the data to the list
    data_list.append(data)

#creating an empty dataframe
df = pd.DataFrame()

#creating a loop to append the data to the dataframe
for i in range(len(data_list)):
    
    #creating an empty list to store the records
    records = []

    #looping through the records
    for record in data_list[i]['records']:
        #storing the records in the list
        records.append(record['fields'])

    #creating a dataframe from the json
    df_temp = pd.DataFrame(records)
    #appending the dataframe to the main dataframe
    df = pd.concat([df, df_temp])

## Transforming data of df

In [5]:
# transformation

#transform the label to numeric
df["label"] = pd.to_numeric(df["label"])
df["annee"] = pd.to_numeric(df["annee"])

#sort annee by ascending order
df.sort_values(by="annee",inplace=True)

#create two new columns for longitude and latitude
df['latitude'] = df['position'].apply(lambda x: float(x[0]))
df['longitude'] = df['position'].apply(lambda x: float(x[1]))

# csv des résultats des lycee professionnels

## import data from api

In [6]:
#creating an empty list
data_list = []

#creating a loop to get the data from all the available pages
for i in range(0, 10000, 5000):
    #creating the url
    url = 'https://data.occitanie.education.gouv.fr/api/records/1.0/search/?dataset=fr-en-indicateurs-de-resultat-des-lycees-denseignement-professionnels&q=&rows=10000&facet=etablissement&facet=code_etablissement&facet=annee&facet=ville&facet=academie&facet=departement&facet=secteur_public_1_prive_2&facet=libelle_region_2016'
    #making the request
    response = requests.get(url)
    #converting the response to json
    data = response.json()
    #appending the data to the list
    data_list.append(data)

#creating an empty dataframe
df2 = pd.DataFrame()

#creating a loop to append the data to the dataframe
for i in range(len(data_list)):
    
    #creating an empty list to store the records
    records = []

    #looping through the records
    for record in data_list[i]['records']:
        #storing the records in the list
        records.append(record['fields'])

    #creating a dataframe from the json
    df_temp = pd.DataFrame(records)
    #appending the dataframe to the main dataframe
    df2 = pd.concat([df, df_temp])

## Nettoyage

In [7]:
# nettoyage

# Le lycee "LYCEE PROFESSIONNEL CASTELNOUVEL" a un taux de réussite brut 0
# Il a aussi un effectif_presents_total_secteurs 1, ce qui errone la visualisation (un point très écarté)
# ainsi qu'un taux_mentions_attendu_tous_secteurs Nan
# j'ai donc décidé d'enlever ce lycee
df2.drop(df2[df2['code_etablissement'] == "0312063Z"].index, inplace = True)


## Transform

In [42]:
# transformation

#transform the annee to numeric
df2["annee"] = pd.to_numeric(df2["annee"])
#sort annee by ascending order
df2.sort_values(by="annee",inplace=True)
# re transform annee to str so it can be a discrete value for visualisations (discrete value)
df2["annee"] = df2["annee"].apply(str)

# ransform le taux brut en numerique
df2["taux_brut_de_reussite_total_secteurs"] = pd.to_numeric(df2["taux_brut_de_reussite_total_secteurs"])


# Operations sur les deux csv (merge)

In [43]:
# merge 2 dataframes on the uri (id de l'etablissement scolaire)
df_result = pd.merge(df, df2, left_on='rne', right_on='code_etablissement', how="inner")

## Nettoyage

In [44]:
#nettoyage (supression) des colonnes ou toutes les valeures sont nulles
df_result.dropna(axis=1,how='all',inplace=True)

#nettoyage (supression) des lignes ou toutes les valeures sont nulles
df_result.dropna(axis=0,how='all',inplace=True)

# L'index 1393 a une valeure ajoutée de réusiite totale = "ND"
# par contre ce lycee a un taux_brut_de_reussite_total_secteurs = 84
# j'ai donc décidé de rempace le "ND" par 0 (neutre) pour ce lycee et pour tous les lycee qui ont
# une valeure ajoutée de réussite totale = "ND" (non définir)
import re
aNegliger = re.compile("ND",re.I)
subst = "0"
def apply_reg(str_):
    '''Fonction pour remplacer les "ND" par 0 tout court dans une colonne
        - paramètres : 
            - str_ : la chaine de caractère dans laquelle la fonction va chercher la pattern à nettoyé
        - Return la chaine de caractère nettoyé '''
    if pd.notna(str_):
        return re.sub(aNegliger, subst, str(str_))
    else:
        return str_
    
df_result['va_reu_total'] = df_result['va_reu_total'].apply(apply_reg)

# Transform

In [79]:
# transformer les dates en numériques pour la prochaine opération
df_result["annee_x"] = pd.to_numeric(df_result["annee_x"])
df_result["annee_y"] = pd.to_numeric(df_result["annee_y"])

# ajouter une colonne qui contient boolean (resultat après obtention label ou pas)
# True si oui, false sinon
# les resultats des lycee après l'obtention de leurs labels 
# (annee_x pour obtention label, annee_y pour passage examens de baccalauréat)
df_result["resultat_apres_label"] = (df_result["annee_y"] >= df_result["annee_x"])

# transformer la valeur ajoutée de réussites totale en numérique
df_result["va_reu_total"] = pd.to_numeric(df_result["va_reu_total"])

# transorm taux_reussite_attendu_france_total_secteurs to numeric
df_result["taux_reussite_attendu_france_total_secteurs"] = pd.to_numeric(df_result["taux_reussite_attendu_france_total_secteurs"])

## Infos

In [80]:
df_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3757 entries, 3756 to 2983
Columns: 136 entries, label_x to resultat_apres_label
dtypes: bool(1), float64(20), int64(3), object(112)
memory usage: 3.9+ MB


In [81]:
for c in df_result.columns:
    print(c)

label_x
annee_x
localite_x
departement_x
nom_etablissement_x
position_x
rne_x
latitude_x
longitude_x
annee_y
departement_y
code_departement
va_men_meca_elec
effectif_presents_services
code_etablissement
effectifs_presents_echanges_et_gestion
taux_brut_de_reussite_mecanique_electricite_electronique
va_acc_terminale
va_men_serv_pers
taux_brut_de_reussite_echanges_et_gestion
libelle_region_2016
taux_brut_de_reussite_services_aux_personnes
taux_brut_de_mentions_services_aux_personnes
structure_pedagogique_7_groupes
va_men_echanges
va_reu_production
taux_brut_de_mentions_echanges_et_gestion
taux_brut_de_reussite_production
academie
taux_acces_brut_seconde_bac
va_reu_serv_pers
va_men_total
va_acc_premiere
va_reu_echanges
taux_brut_de_mentions_mecanique_electricite_electronique
libelle_departement
taux_brut_de_reussite_services
va_men_transfo
effectifs_presents_mecanique_electricite_electronique
taux_brut_de_reussite_total_secteurs
effectifs_presents_services_aux_personnes
va_reu_transfo
effe

# Transform for visualisations

In [82]:
# re transform annee to str so it can be a discrete value for visualisations (discrete value)
df_result["annee_x"] = df_result["annee_x"].apply(str)
df_result["annee_y"] = df_result["annee_y"].apply(str)



# Visualisation 

In [59]:
# Import necessary libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import folium # for maps

## Analyse du taux brut de réussite des élèves dans les lycées

**Taux de réussite constaté**

- Le taux de réussite au baccalauréat est la proportion, parmi les élèves présents à l’examen, de ceux qui ont obtenu le diplôme
- Le graphique ci-dessous montre que le moyenne du taux de réussite est presque similaire selon que l'école
    a un abel 1 ou 3.
- Il est donc intéréssant d'aller voir qu'elle étaient les moyennes de taux de réussite de ces écoles avant
    obtention de leurs labels numérique !

In [77]:
# transform label to numeric
df_result["label_x"] = pd.to_numeric(df_result["label_x"])
#sort label by ascending order
df_result.sort_values(by="label_x",inplace=True)
# re transform label to str so it can be a discrete value for visualisations (discrete value)
df_result["label_x"] = df_result["label_x"].apply(str)

#plotting a histogram 
fig = px.histogram(df_result[df_result["resultat_apres_label"] == True], x="label_x", y="taux_brut_de_reussite_total_secteurs", nbins=10, histfunc="avg", color="label_x")
fig.update_layout(title_text="La moyenne des taux bruts de réussites des lycee après obtention de leurs labels (tous secteurs) selon les labels")
fig.show()

**Comparaison de la moyenne du taux de réussite constaté avant et après obtention des labels numériques**

- Le graphique ci-dessous montre une amélioration des taux de réussite des écoles après obtention de leurs labels
    numérique !
- IL est donc intéréssant d'aller chercher pourquoi ? est ce que l'école a apporté plus de valeur pour les 
    élèves grace au numérique ?

In [88]:
#plotting a histogram 
fig = px.histogram(df_result, x="departement_y", y="taux_brut_de_reussite_total_secteurs", nbins=10,
                       histfunc="avg", color="departement_y",
                       facet_row="resultat_apres_label",
                      labels={"taux_brut_de_reussite_total_secteurs" : "taux brut de réussite"})
fig.update_layout(title_text="La moyenne des taux bruts de réussites (tous secteurs) selon les départements - avant et après obtention labels")
fig.show()

In [89]:
#plotting a histogram 
fig = px.histogram(df_result, x="rne_x", y="taux_brut_de_reussite_total_secteurs", nbins=10, histfunc="avg", color="departement_y", facet_row="resultat_apres_label",
                  labels={"taux_brut_de_reussite_total_secteurs" : "taux brut de réussite"})
fig.update_layout(title_text="La moyenne des taux bruts de réussites de chaque lycée (tous secteurs) avant et après obtention labels")
fig.show()

**Pourquoi certains lycées n'apparaissent pas dans le resultat après labels malgré qu'ils ont des labels ?**
- simplement parce ces lycées n'ont pas eu de passages d'examens après obtentions de labels (peut etre étrange mais 
    voici les données suivantes comme exemple pour l'école 0480009Z

In [71]:

df_result[df_result['rne_x'] == "0480009Z"][["label_x","annee_x","annee_y","code_etablissement","resultat_apres_label"]]

,label_x,annee_x,annee_y,code_etablissement,resultat_apres_label
836,1,2017,2016,0480009Z,False
837,1,2018,2016,0480009Z,False
838,1,2019,2016,0480009Z,False
839,1,2021,2016,0480009Z,False


## Analyse de la valeur ajoutée de réussite 
- la valeur ajoutée de l’établissement est la différence entre le taux constaté de l’établissement et le taux attendu. 

    **Valeur ajoutée = taux constaté – taux attendu**
    

- Le graphique ci-dessous montre une moyenne de valeur ajoutée de label 3 plus petite que label 1.
    C'est étonnant, mais est-ce que ça veut dire que les écoles ayant un label 3 ont contribué moins
    à la réussite des élèves ? Pas nécessairement.
- En effet, peut etre que les écoles ayant un label 3, ont un taux de réussite attendu beaucoup plus elevé 
    que les écoles avec un label plus faible. De ce fait, l'augmentation du taux de réussite attendu 
    peut diminuer la valeur ajoutée de l'école.

In [78]:
# transform label to numeric
df_result["label_x"] = pd.to_numeric(df_result["label_x"])
#sort label by ascending order
df_result.sort_values(by="label_x",inplace=True)
# re transform label to str so it can be a discrete value for visualisations (discrete value)
df_result["label_x"] = df_result["label_x"].apply(str)

#plotting a histogram 
fig = px.histogram(df_result[df_result["resultat_apres_label"] == True], x="label_x", y="va_reu_total", nbins=10, histfunc="avg", color="label_x")
fig.update_layout(title_text="La moyenne des valeurs ajoutées de réussites (tous secteurs) selon les labels")
fig.show()

**Comparaison des valeurs ajoutées des écoles avant et après obtention des labels numériques**

- le graphique ci-dessous montre une véritables amélioration des valeurs ajoutée des écoles après obtention de labels. Surtout pour le département de LOZERE pour lequel la valeur ajoutée est passée du **-** à **+** !

In [90]:
#plotting a histogram 
fig = px.histogram(df_result, x="departement_y", y="va_reu_total", nbins=10, histfunc="avg", color="departement_y", facet_row="resultat_apres_label")
fig.update_layout(title_text="La moyenne des valeurs ajoutés de réussites (tous secteurs) selon les départements avant et après obtention labels")
fig.show()

## Analyse du taux de réussite attendu en france (total secteurs)

In [91]:
# transform label to numeric
df_result["label_x"] = pd.to_numeric(df_result["label_x"])
#sort label by ascending order
df_result.sort_values(by="label_x",inplace=True)
# re transform label to str so it can be a discrete value for visualisations (discrete value)
df_result["label_x"] = df_result["label_x"].apply(str)

#plotting a histogram 
fig = px.histogram(df_result[df_result["resultat_apres_label"] == True], x="label_x", y="taux_reussite_attendu_france_total_secteurs", nbins=10, histfunc="avg", color="label_x")
fig.update_layout(title_text="La moyenne des taux de réussite attendu en france (tous secteurs) selon les labels")
fig.show()

In [92]:
#plotting a histogram 
fig = px.histogram(df_result, x="departement_y", y="taux_reussite_attendu_france_total_secteurs", nbins=10,
                       histfunc="avg", color="departement_y",
                       facet_row="resultat_apres_label",
                      labels={"taux_reussite_attendu_france_total_secteurs" : "Taux_réussite_attendu"})
fig.update_layout(title_text="La moyenne des taux de réussite attendu en france (tous secteurs) selon les départements - avant et après obtention labels")
fig.show()